In [1]:
%cd /work/tomishima2904/explore_conceptnet
!ls

/work/tomishima2904/explore_conceptnet
Dockerfile  do_1.sh  docker-compose.yml  nohup.out	   src
datasets    do_2.sh  logfile.log	 requirements.txt  tmp
do_0.sh     do_3.sh  logs		 results	   venv


In [2]:
import csv
import json
import random

# 様々な条件でテンプレートを生成

## 自動生成された連想理由を使用したテンプレート作成

In [3]:
# 自動生成された連想理由のデータ読み込み
input_path = "results/ja/連想語頻度表/text_generation/231031120400_dev30_M10_0S_0R/m_and_a/len/diffs_btween_manda_formatted6.csv"
with open(input_path, 'r') as f:
    reader = csv.reader(f)
    input_data_a = [row for row in reader]

In [4]:
# n-shot，被りなし（deduplicated）
n = 10
sample_data = [[*row[:3], eval(row[3])[0], eval(row[4])[0], eval(row[5])[0], eval(row[6])[0], eval(row[7])[0]] for row in input_data_a[:n]]
sample_data_a = [row for row in sample_data[:5]]
sample_data_a

[['/r/IsA',
  'ピアノ',
  '楽器',
  '(1) ピアノは楽器の一種なので、連想が成り立つ',
  (0, 0),
  (1.0, 1.0),
  (0.13293825089931488, 0.2503490149974823),
  3],
 ['/r/AtLocation',
  'イルカ',
  '水族館',
  'イルカが水族館で展示される事が多いなど',
  (1, 0),
  (0.5, 1.0),
  (0.08893381059169769, 0.2101650834083557),
  3],
 ['/r/IsA',
  'テニス',
  'スポーツ',
  'テニスはスポーツの一つである',
  (2, 0),
  (0.3333333333333333, 1.0),
  (0.08271581679582596, 0.8463014960289001),
  3],
 ['/r/MadeOf',
  'うどん',
  '小麦粉',
  'うどんは小麦粉を使って作られるから',
  (10, 0),
  (0.09090909090909091, 1.0),
  (0.013872388750314713, 0.7547534108161926),
  3],
 ['/r/Synonym',
  '空気',
  '大気',
  '空気は酸素と水と窒素からなるなど、大気は大気中で起こる現象の名前などから連想される',
  (2, 0),
  (0.3333333333333333, 1.0),
  (0.054228588938713074, 0.5421711206436157),
  0]]

In [32]:
# 2n-shot，best10（decuplicated）
n = 10
assert n % 2 == 0
sample_data = []
for row in input_data_a[:n//2]:
    sample_data.append([*row[:3], eval(row[3])[0], eval(row[4])[0], eval(row[5])[0], eval(row[6])[0], eval(row[7])[0]])
    sample_data.append([*row[:3], eval(row[3])[1], eval(row[4])[1], eval(row[5])[1], eval(row[6])[1], eval(row[7])[1]])

In [ ]:
# n-shot 被りあり（duplicated）
n = 5
flatten_data = [[*row[:3], cpl, ranks, rrs, scores, label] for row in input_data_a
                for cpl, ranks, rrs, scores, label in zip(eval(row[3]), eval(row[4]), eval(row[5]), eval(row[6]), eval(row[7]))]
flatten_data.sort(key=lambda x: (x[5][0]+x[5][1], 1/len(x[3]), x[6][0]+x[6][1]), reverse=True)
sample_data = flatten_data[:n]
sample_data

In [27]:
# n-shot，被り無し，ランダム
# ランダムだけど前処理はする（completionの中に刺激語と連想語が含まれているものの中からランダムに選ぶ）
n = 5
sample_data = []
label_flag = 1  # 2 or 0

seed_value = 12
random.seed(seed_value)

for row in input_data_a[:n]:
    cpl_list = []
    ranks_list = []
    rrs_list = []
    scores_list = []
    label_list = []
    for cpl, ranks, rrs, scores, label in zip(eval(row[3]), eval(row[4]), eval(row[5]), eval(row[6]), eval(row[7])):
        if (rrs[0] >= 0) and (rrs[1] >= 0):
            cpl_list.append(cpl)
            ranks_list.append(ranks)
            rrs_list.append(rrs)
            scores_list.append(scores)
            label_list.append(label)
        else:
            continue

    random_id = random.randint(0, len(cpl_list)-1)
    # labelが全て1 or 0のものでランダムに選定したい場合
    # if any(label_list):
    #     while True:
    #         random_id = random.randint(0, len(cpl_list)-1)
    #         if label_list[random_id] > 1:
    #             break
    # else:
    #     print("Error")
    #     random_id = random.randint(0, len(cpl_list)-1)
    sample_data.append([*row[:3], cpl_list[random_id], ranks_list[random_id], rrs_list[random_id], scores_list[random_id], label_list[random_id]])
# sample_data

In [28]:
# 自動生成された連想理由
input_path = "results/ja/連想語頻度表/text_generation/231031120400_dev30_M10_0S_0R/m_and_a/len/diffs_btween_manda_formatted6.csv"
output_path = f"datasets/連想語頻度表/templates/{n}-shot_len_deduplicated_random/{n}-shot_len_deduplicated_random_{seed_value}.json"

templates_dict = \
{
    "description": f"2023/12/7 項目間5-shotで項目間被りなし，ランダム，シード値{seed_value}",
    "prompt_input": "与えられた入力語から出力語が連想される理由について解答しなさい。\n",
    "replace_template": "",
    "sample_data": sample_data,
}

template = "入力語:{head} 出力語:{tail}\n連想の理由:{completion}"

for row in sample_data:
    tmp_template = template
    head = row[1]
    tail = row[2]
    completion = row[3]
    tmp_template = tmp_template.replace("{head}", head)
    tmp_template = tmp_template.replace("{tail}", tail)
    tmp_template = tmp_template.replace("{completion}", completion)
    templates_dict["prompt_input"] += f"{tmp_template}\n\n"
templates_dict["prompt_input"] += "入力語:{head} 出力語:{tail}\n連想の理由:"
templates_dict["replace_template"] = templates_dict["prompt_input"].replace("\n", " ")

print(templates_dict["prompt_input"])

with open(output_path, 'w') as f:
    json.dump(templates_dict, f, ensure_ascii=False, indent=2)

与えられた入力語から出力語が連想される理由について解答しなさい。
入力語:ピアノ 出力語:楽器
連想の理由:ピアノは子どもの頃から好きで、楽器は音楽を作るための道具です

入力語:イルカ 出力語:水族館
連想の理由:水族館は、海にある施設で、イルカが水槽に入っている姿を見ることができる施設

入力語:テニス 出力語:スポーツ
連想の理由:連想語リストを見ると、スポーツに「テニス」が含まれていないのに、「テニス」が「スポ

入力語:うどん 出力語:小麦粉
連想の理由:うどんを食べたことがある人は小麦粉を食べたことがある可能性が高い

入力語:空気 出力語:大気
連想の理由: 空気は大気の一部である

入力語:{head} 出力語:{tail}
連想の理由:


## 手動生成された連想理由を使用したテンプレート作成

In [5]:
# 手動生成された連想理由のデータを読み込み
input_path = "datasets/連想語頻度表/all/dev_mini2/htrkpns_tmp_manual_RTY.csv"
with open(input_path, 'r') as f:
    reader = csv.reader(f)
    input_data_m = [row for row in reader]

In [7]:
# 特定の個人の連想理由を使用
subject_id = 0  # 0: 富島, 1: 問井, 2: 悠真
n = 10
cpl_id_list = [9, 8, 10, 13, 22, 4, 2, 19, 12, 14]
assert len(cpl_id_list) == n, f"shot数を適切に揃えてください"

sample_data = [[eval(row[2])[0], *row[:2], eval(row[6])[subject_id]] for row in input_data_m]
sample_data = [sample_data[i-1] for i in cpl_id_list]

In [8]:
# ランダムな連想理由を使用
n = 10
cpl_id_list = [9, 8, 10, 13, 22, 4, 2, 19, 12, 14]
assert len(cpl_id_list) == n, f"shot数を適切に揃えてください"

sample_data = [[eval(row[2])[0], *row[:2], eval(row[6])] for row in input_data_m]
sample_data = [[*sample_data[i-1][:3], random.choice(sample_data[i-1][3])] for i in cpl_id_list]
print(sample_data)
sample_data_m = [row for row in sample_data[5:]]
sample_data_m

[['/r/IsA', 'ピアノ', '楽器', 'ピアノは楽器の1つだからです'], ['/r/IsA', 'イルカ', '哺乳類', 'イルカは哺乳類に属しているからです'], ['/r/IsA', 'テニス', 'スポーツ', 'テニスはスポーツの仲間だからです'], ['/r/MadeOf', 'うどん', '小麦粉', 'うどんは小麦粉からできているからです'], ['/r/Synonym', '空気', '大気', '空気は大気とも言われるからです'], ['/r/AtLocation', 'カジノ', 'ラスベガス', 'ラスベガスがカジノの聖地だからです'], ['/r/AtLocation', 'イルカ', '水族館', 'イルカがいる水族館があるからです'], ['/r/PartOf', '画面', 'パソコン', 'パソコンの一部に画面があるからです'], ['/r/MadeOf', 'ビール', '麦', 'ビールは麦で作られているからです'], ['/r/MadeOf', 'ワイン', 'アルコール', 'ワインはアルコールの一種だからです']]


[['/r/AtLocation', 'カジノ', 'ラスベガス', 'ラスベガスがカジノの聖地だからです'],
 ['/r/AtLocation', 'イルカ', '水族館', 'イルカがいる水族館があるからです'],
 ['/r/PartOf', '画面', 'パソコン', 'パソコンの一部に画面があるからです'],
 ['/r/MadeOf', 'ビール', '麦', 'ビールは麦で作られているからです'],
 ['/r/MadeOf', 'ワイン', 'アルコール', 'ワインはアルコールの一種だからです']]

In [11]:
# 全員使用
n = 15
num_subjects = 3
cpl_id_list = [9, 8, 10, 13, 22]

sample_data_ = [[eval(row[2])[0], *row[:2], eval(row[6])] for row in input_data_m]
sample_data_ = [sample_data_[i-1] for i in cpl_id_list]
sample_data = []
for row in sample_data_:
    for i in range(num_subjects):
        sample_data.append([*row[:-1], row[-1][i]])

sample_data

[['/r/IsA', 'ピアノ', '楽器', 'ピアノは楽器の1つだからです'],
 ['/r/IsA', 'ピアノ', '楽器', 'ピアノは楽器の一種だからです'],
 ['/r/IsA', 'ピアノ', '楽器', 'ピアノは楽器の一つだからです'],
 ['/r/IsA', 'イルカ', '哺乳類', 'イルカは哺乳類に属しているからです'],
 ['/r/IsA', 'イルカ', '哺乳類', 'イルカは哺乳類の仲間だからです'],
 ['/r/IsA', 'イルカ', '哺乳類', 'イルカは哺乳類だからです'],
 ['/r/IsA', 'テニス', 'スポーツ', 'テニスはスポーツの1つだからです'],
 ['/r/IsA', 'テニス', 'スポーツ', 'テニスはスポーツの仲間だからです'],
 ['/r/IsA', 'テニス', 'スポーツ', 'テニスはスポーツの一つだからです'],
 ['/r/MadeOf', 'うどん', '小麦粉', 'うどんは小麦粉からできているからです'],
 ['/r/MadeOf', 'うどん', '小麦粉', 'うどんは小麦粉からできているからです'],
 ['/r/MadeOf', 'うどん', '小麦粉', 'うどんは小麦粉で作られているからです'],
 ['/r/Synonym', '空気', '大気', '空気と大気は同じだからです'],
 ['/r/Synonym', '空気', '大気', '空気は大気に含まれるからです'],
 ['/r/Synonym', '空気', '大気', '空気は大気とも言われるからです']]

In [7]:
# 自動と手動のmix
n = 10
sample_data_mix = []
for row_a, row_m in zip(sample_data_a, sample_data_m):
    sample_data_mix.append(row_a)
    sample_data_mix.append(row_m)

sample_data = [row for row in sample_data_mix]
random.shuffle(sample_data)
sample_data

[['/r/AtLocation', 'イルカ', '水族館', 'イルカがいる水族館があるからです'],
 ['/r/Synonym',
  '空気',
  '大気',
  '空気は酸素と水と窒素からなるなど、大気は大気中で起こる現象の名前などから連想される',
  (2, 0),
  (0.3333333333333333, 1.0),
  (0.054228588938713074, 0.5421711206436157),
  0],
 ['/r/AtLocation', 'カジノ', 'ラスベガス', 'カジノはラスベガスが有名だからです'],
 ['/r/IsA',
  'テニス',
  'スポーツ',
  'テニスはスポーツの一つである',
  (2, 0),
  (0.3333333333333333, 1.0),
  (0.08271581679582596, 0.8463014960289001),
  3],
 ['/r/IsA',
  'ピアノ',
  '楽器',
  '(1) ピアノは楽器の一種なので、連想が成り立つ',
  (0, 0),
  (1.0, 1.0),
  (0.13293825089931488, 0.2503490149974823),
  3],
 ['/r/PartOf', '画面', 'パソコン', 'パソコンには画面がついているからです'],
 ['/r/MadeOf', 'ビール', '麦', 'ビールは麦からできているからです'],
 ['/r/MadeOf',
  'うどん',
  '小麦粉',
  'うどんは小麦粉を使って作られるから',
  (10, 0),
  (0.09090909090909091, 1.0),
  (0.013872388750314713, 0.7547534108161926),
  3],
 ['/r/MadeOf', 'ワイン', 'アルコール', 'ワインはアルコールを含んでいるからです'],
 ['/r/AtLocation',
  'イルカ',
  '水族館',
  'イルカが水族館で展示される事が多いなど',
  (1, 0),
  (0.5, 1.0),
  (0.08893381059169769, 0.2101650834083557),
  3]]

In [9]:
# 手動生成された連想理由
input_path = "results/ja/連想語頻度表/text_generation/231031120400_dev30_M10_0S_0R/m_and_a/len/diffs_btween_manda_formatted6.csv"
output_path = f"datasets/連想語頻度表/templates/{n}-shot_len_xRandom.json"

templates_dict = \
{
    "description": "2023/11/30 項目間5-shotで項目間被りなし (ランダム)",
    "prompt_input": "与えられた入力語から出力語が連想される理由について解答しなさい。\n",
    "replace_template": "",
    "sample_data": sample_data,
}

template = "入力語:{head} 出力語:{tail}\n連想の理由:{completion}"

for row in sample_data:
    tmp_template = template
    head = row[1]
    tail = row[2]
    completion = row[3]
    tmp_template = tmp_template.replace("{head}", head)
    tmp_template = tmp_template.replace("{tail}", tail)
    tmp_template = tmp_template.replace("{completion}", completion)
    templates_dict["prompt_input"] += f"{tmp_template}\n\n"
templates_dict["prompt_input"] += "入力語:{head} 出力語:{tail}\n連想の理由:"
templates_dict["replace_template"] = templates_dict["prompt_input"].replace("\n", " ")

print(templates_dict["prompt_input"])

with open(output_path, 'w') as f:
    json.dump(templates_dict, f, ensure_ascii=False, indent=2)

与えられた入力語から出力語が連想される理由について解答しなさい。
入力語:ピアノ 出力語:楽器
連想の理由:ピアノは楽器の1つだからです

入力語:イルカ 出力語:哺乳類
連想の理由:イルカは哺乳類に属しているからです

入力語:テニス 出力語:スポーツ
連想の理由:テニスはスポーツの仲間だからです

入力語:うどん 出力語:小麦粉
連想の理由:うどんは小麦粉からできているからです

入力語:空気 出力語:大気
連想の理由:空気は大気とも言われるからです

入力語:カジノ 出力語:ラスベガス
連想の理由:ラスベガスがカジノの聖地だからです

入力語:イルカ 出力語:水族館
連想の理由:イルカがいる水族館があるからです

入力語:画面 出力語:パソコン
連想の理由:パソコンの一部に画面があるからです

入力語:ビール 出力語:麦
連想の理由:ビールは麦で作られているからです

入力語:ワイン 出力語:アルコール
連想の理由:ワインはアルコールの一種だからです

入力語:{head} 出力語:{tail}
連想の理由:


In [38]:
output_path = "datasets/連想語頻度表/templates/few-shot_no_refs_6.json"
with open(output_path, 'w') as f:
    json.dump(templates_dict, f, ensure_ascii=False, indent=2)